<a href="https://colab.research.google.com/github/ereshmittal/Article-Spinner/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot

import textwrap
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2022-12-01 17:18:14--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  8.73MB/s    in 0.6s    

2022-12-01 17:18:15 (8.73 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [ ]:
df = pd.read_csv('bbc_text_cls.csv')
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [ ]:
df.labels.unique()

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype=object)

In [ ]:
i = np.random.randint(0, len(df))
doc = df.iloc[i]

In [ ]:
print(textwrap.fill(doc['text'], replace_whitespace=False, fix_sentence_endings=True, width=120))

Oasis star fined for German brawl

Oasis singer Liam Gallagher has been fined 50,000 euros (£35,000) after a fight in a
German hotel two years ago.

Gallagher was arrested along with drummer Alan White and three other members of the band's
entourage after the brawl in Munich in December 2002. The band said they were victims of an "unprovoked attack" in a
nightclub.  But police said Gallagher kicked an officer in the chest and had large amounts of alcohol and drugs -
possibly cocaine - in his blood.  Gallagher lost two front teeth in the fight, which led to the band abandoning their
German tour.  His brother and bandmate Noel was in bed at the time.

"The process has stopped by paying 50,000 euros,"
said Anton Winkler, spokesman for the Munich prosecutor.  At the time, police said a "physical altercation" broke out
among the musicians at about 0200 local time.  That led to one of the group being "jostled" and falling onto the table
of five Italian guests - causing the fight, they said. 

In [ ]:
sents = nltk.sent_tokenize(doc['text'].split("\n", 1)[1])

In [ ]:
vectorizer = TfidfVectorizer(stop_words = stopwords.words('english'), norm='l1')

In [ ]:
X = vectorizer.fit_transform(sents)

In [ ]:
X

<10x87 sparse matrix of type '<class 'numpy.float64'>'
	with 114 stored elements in Compressed Sparse Row format>

In [ ]:
def get_sent_score(tfidf_row):
  return tfidf_row[tfidf_row!=0].mean()

In [ ]:
scores = np.zeros(len(sents))
for i in range(len(sents)):
  score = get_sent_score(X[i])
  scores[i] = score

In [ ]:
scores

array([0.06666667, 0.07142857, 0.16666667, 0.07692308, 0.09090909,
       0.2       , 0.08333333, 0.1       , 0.07692308, 0.06666667])

In [ ]:
sort_idx = np.argsort(-scores)

In [ ]:
for i in sort_idx[:5]:
  print(textwrap.wrap(f"sent: {sents[i]}, score: {scores[i]}"))

['sent: His brother and bandmate Noel was in bed at the time., score:', '0.19999999999999998']
['sent: The band said they were victims of an "unprovoked attack" in a', 'nightclub., score: 0.16666666666666666']
['sent: At the time, police said a "physical altercation" broke out', 'among the musicians at about 0200 local time., score:', '0.09999999999999999']
['sent: Gallagher lost two front teeth in the fight, which led to the', 'band abandoning their German tour., score: 0.09090909090909091']
['sent: "The process has stopped by paying 50,000 euros," said Anton', 'Winkler, spokesman for the Munich prosecutor., score:', '0.08333333333333331']


In [ ]:
doc.iloc[0].split('\n', 1)[0]

'Oasis star fined for German brawl'